In [2]:
import open3d as o3d
import numpy as np
from typing import Dict, List, Optional
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

class MLIntentClassifier:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.classifier = LinearSVC()
        self.intents = ['furniture', 'lighting', 'storage', 'decorative']
        
        # Mock training data (replace with real dataset)
        mock_data = [
            ("table with legs", 'furniture'),
            ("lamp with shade", 'lighting'),
            ("ornamental sculpture", 'decorative'),
            ("bookshelf with drawers", 'storage')
        ]
        texts, labels = zip(*mock_data)
        X = self.vectorizer.fit_transform(texts)
        self.classifier.fit(X, labels)

    def predict(self, text: str) -> str:
        X = self.vectorizer.transform([text])
        return self.classifier.predict(X)[0]

class ShapeGenerator:
    @staticmethod
    def create_shape(shape_type: str, params: Dict) -> o3d.geometry.TriangleMesh:
        if shape_type == 'box':
            mesh = o3d.geometry.TriangleMesh.create_box(
                width=params['width'],
                height=params['height'],
                depth=params['depth']
            )
        elif shape_type == 'cylinder':
            mesh = o3d.geometry.TriangleMesh.create_cylinder(
                radius=params['radius'],
                height=params['height'],
                resolution=params.get('resolution', 32)
            )
        elif shape_type == 'sphere':
            mesh = o3d.geometry.TriangleMesh.create_sphere(
                radius=params['radius'],
                resolution=params.get('resolution', 20)
            )
        elif shape_type == 'cone':
            # Create cone by scaling cylinder
            mesh = o3d.geometry.TriangleMesh.create_cylinder(
                radius=params['base_radius'],
                height=params['height'],
                resolution=params.get('resolution', 32)
            )
            vertices = np.asarray(mesh.vertices)
            for i in range(len(vertices)):
                h_ratio = vertices[i][2] / params['height']
                vertices[i][0] *= (1 - h_ratio)
                vertices[i][1] *= (1 - h_ratio)
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        elif shape_type == 'torus':
            mesh = ShapeGenerator._create_torus(
                major_radius=params['radius'],
                minor_radius=params['section_radius'],
                resolution=params.get('resolution', 32)
            )
        else:
            raise ValueError(f"Unsupported shape type: {shape_type}")
            
        mesh.compute_vertex_normals()
        return mesh

    @staticmethod
    def _create_torus(major_radius: float, minor_radius: float, resolution: int = 32) -> o3d.geometry.TriangleMesh:
        u = np.linspace(0, 2*np.pi, resolution)
        v = np.linspace(0, 2*np.pi, resolution)
        U, V = np.meshgrid(u, v)
        
        x = (major_radius + minor_radius*np.cos(V)) * np.cos(U)
        y = (major_radius + minor_radius*np.cos(V)) * np.sin(U)
        z = minor_radius * np.sin(V)
        
        vertices = np.vstack((x.flatten(), y.flatten(), z.flatten())).T
        
        # Create faces
        faces = []
        for i in range(resolution-1):
            for j in range(resolution-1):
                idx = i * resolution + j
                faces.extend([
                    [idx, idx+1, idx+resolution],
                    [idx+1, idx+resolution+1, idx+resolution]
                ])
                
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(vertices)
        mesh.triangles = o3d.utility.Vector3iVector(faces)
        return mesh

class Open3DGenerator:
    def __init__(self):
        self.components = []
        self.reasoning_log = []
        self.intent_classifier = MLIntentClassifier()
        self.shape_generator = ShapeGenerator()
        
        self.default_params = {
            'width': 1.0,
            'depth': 1.0,
            'height': 1.0,
            'radius': 0.5,
            'section_radius': 0.1,
            'resolution': 32,
            'base_radius': 0.5
        }

    def log_step(self, message: str):
        self.reasoning_log.append(f"• {message}")

    def parse_description(self, text: str):
        self.log_step(f"Analyzing description: '{text}'")
        text = text.lower()
        
        intent = self.intent_classifier.predict(text)
        self.log_step(f"ML-classified intent: {intent.upper()}")
        
        if intent == 'furniture':
            self.default_params.update({'width': 2.0, 'height': 0.8})
        elif intent == 'lighting':
            self.default_params.update({'radius': 0.3, 'height': 1.5})

        shape = self._detect_shape(text)
        self.log_step(f"Detected primary shape: {shape}")
        
        self.components.append({
            'type': 'main_body',
            'shape': shape,
            'transform': np.eye(4)
        })

        self._detect_supports(text)

    def _detect_shape(self, text: str) -> str:
        shape_keywords = {
            'spherical': 'sphere',
            'round': 'cylinder',
            'conical': 'cone',
            'pyramid': 'cone',
            'ring': 'torus',
            'angular': 'box'
        }
        for kw, shape in shape_keywords.items():
            if kw in text:
                return shape
        return 'box'

    def _detect_supports(self, text: str):
        support_types = {
            'legs': {'shape': 'cylinder', 'params': {'radius': 0.05}},
            'base': {'shape': 'cylinder', 'params': {'radius': 0.3, 'height': 0.1}},
        }
        
        for kw, config in support_types.items():
            if kw in text:
                count = next((int(n) for n in text.split() if n.isdigit()), 4)
                self.log_step(f"Detected {count} {kw} using {config['shape']} shape")
                
                if count > 1:
                    angle_step = 2 * np.pi / count
                    radius = 0.3  # Distance from center
                    
                    for i in range(count):
                        angle = i * angle_step
                        transform = np.eye(4)
                        transform[0:3, 3] = [radius * np.cos(angle), radius * np.sin(angle), 0]
                        
                        self.components.append({
                            'type': kw,
                            'shape': config['shape'],
                            'params': config['params'],
                            'transform': transform
                        })
                else:
                    self.components.append({
                        'type': kw,
                        'shape': config['shape'],
                        'params': config['params'],
                        'transform': np.eye(4)
                    })

    def generate_component(self, part: Dict) -> o3d.geometry.TriangleMesh:
        self.log_step(f"Generating {part['type']} ({part['shape']})")
        
        params = self.default_params.copy()
        params.update(part.get('params', {}))
        
        mesh = self.shape_generator.create_shape(part['shape'], params)
        mesh.transform(part['transform'])
        
        return mesh

    def generate(self, description: str) -> o3d.geometry.TriangleMesh:
        self.components = []
        self.reasoning_log = []
        
        self.parse_description(description)
        
        # Combine all components
        full_mesh = o3d.geometry.TriangleMesh()
        for part in self.components:
            mesh = self.generate_component(part)
            full_mesh += mesh
        
        full_mesh.compute_vertex_normals()
        
        print("\n".join(self.reasoning_log))
        print(f"\n✅ Generated {len(self.components)} components")
        
        return full_mesh

# Example usage
generator = Open3DGenerator()
mesh = generator.generate("A dog made out of 100 shapes")

# Visualize the result
o3d.visualization.draw_geometries([mesh])

• Analyzing description: 'A dog made out of 100 shapes'
• ML-classified intent: DECORATIVE
• Detected primary shape: box
• Generating main_body (box)

✅ Generated 1 components


In [ ]:
import open3d as o3d
import numpy as np
from typing import Dict, List, Optional

class ShapeGenerator:
    @staticmethod
    def create_shape(shape_type: str, params: Dict) -> o3d.geometry.TriangleMesh:
        if shape_type == 'box':
            mesh = o3d.geometry.TriangleMesh.create_box(
                width=params['width'],
                height=params['height'],
                depth=params['depth']
            )
        elif shape_type == 'cylinder':
            mesh = o3d.geometry.TriangleMesh.create_cylinder(
                radius=params['radius'],
                height=params['height'],
                resolution=params.get('resolution', 32)
            )
        elif shape_type == 'sphere':
            mesh = o3d.geometry.TriangleMesh.create_sphere(
                radius=params['radius'],
                resolution=params.get('resolution', 20)
            )
        elif shape_type == 'cone':
            mesh = o3d.geometry.TriangleMesh.create_cylinder(
                radius=params['base_radius'],
                height=params['height'],
                resolution=params.get('resolution', 32)
            )
            vertices = np.asarray(mesh.vertices)
            for i in range(len(vertices)):
                h_ratio = vertices[i][2] / params['height']
                vertices[i][0] *= (1 - h_ratio)
                vertices[i][1] *= (1 - h_ratio)
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        
        mesh.compute_vertex_normals()
        return mesh

class DynamicShapeGenerator:
    def __init__(self):
        self.components = []
        self.shape_generator = ShapeGenerator()
        
    def add_component(self, shape_type: str, params: Dict, transform: np.ndarray):
        self.components.append({
            'shape_type': shape_type,
            'params': params,
            'transform': transform
        })

    def create_dog(self, size: float = 1.0, style: str = 'normal'):
        self.components = []
        
        # Scaling factors based on size
        body_length = size * 1.2
        body_height = size * 0.6
        body_width = size * 0.4
        head_size = size * 0.3
        leg_height = size * 0.4
        leg_radius = size * 0.05
        
        # Body (elongated box)
        body_transform = np.eye(4)
        self.add_component('box', {
            'width': body_width,
            'height': body_height,
            'depth': body_length
        }, body_transform)
        
        # Head (sphere with slightly elongated snout)
        head_transform = np.eye(4)
        head_transform[0:3, 3] = [0, body_height/2, body_length/2 + head_size/2]
        self.add_component('sphere', {
            'radius': head_size,
            'resolution': 20
        }, head_transform)
        
        # Snout (cone)
        snout_transform = np.eye(4)
        snout_transform[0:3, 3] = [0, body_height/2, body_length/2 + head_size + head_size/4]
        self.add_component('cone', {
            'base_radius': head_size/2,
            'height': head_size/2,
            'resolution': 16
        }, snout_transform)
        
        # Ears (2 cones)
        ear_positions = [
            [head_size/2, body_height/2 + head_size, body_length/2 + head_size/2],
            [-head_size/2, body_height/2 + head_size, body_length/2 + head_size/2]
        ]
        for pos in ear_positions:
            ear_transform = np.eye(4)
            ear_transform[0:3, 3] = pos
            self.add_component('cone', {
                'base_radius': head_size/4,
                'height': head_size/2,
                'resolution': 8
            }, ear_transform)
        
        # Legs (4 cylinders)
        leg_positions = [
            [body_width/3, -leg_height/2, body_length/3],    # Front right
            [-body_width/3, -leg_height/2, body_length/3],   # Front left
            [body_width/3, -leg_height/2, -body_length/3],   # Back right
            [-body_width/3, -leg_height/2, -body_length/3]   # Back left
        ]
        for pos in leg_positions:
            leg_transform = np.eye(4)
            leg_transform[0:3, 3] = pos
            self.add_component('cylinder', {
                'radius': leg_radius,
                'height': leg_height,
                'resolution': 12
            }, leg_transform)
        
        # Tail (cylinder with rotation)
        tail_transform = np.eye(4)
        tail_transform[0:3, 3] = [0, body_height/3, -body_length/2]
        rotation_matrix = self._create_rotation_matrix([1, 0, 0], np.pi/4)  # 45-degree rotation
        tail_transform[0:3, 0:3] = rotation_matrix
        self.add_component('cylinder', {
            'radius': leg_radius,
            'height': body_length/2,
            'resolution': 12
        }, tail_transform)

    def _create_rotation_matrix(self, axis: List[float], angle: float) -> np.ndarray:
        """Create a rotation matrix for rotating around an axis by an angle."""
        axis = np.array(axis)
        axis = axis / np.sqrt(np.sum(axis**2))
        a = np.cos(angle/2)
        b, c, d = axis * np.sin(angle/2)
        return np.array([
            [a*a+b*b-c*c-d*d, 2*(b*c-a*d), 2*(b*d+a*c)],
            [2*(b*c+a*d), a*a+c*c-b*b-d*d, 2*(c*d-a*b)],
            [2*(b*d-a*c), 2*(c*d+a*b), a*a+d*d-b*b-c*c]
        ])

    def generate(self) -> o3d.geometry.TriangleMesh:
        full_mesh = o3d.geometry.TriangleMesh()
        
        for component in self.components:
            mesh = self.shape_generator.create_shape(
                component['shape_type'],
                component['params']
            )
            mesh.transform(component['transform'])
            full_mesh += mesh
        
        full_mesh.compute_vertex_normals()
        return full_mesh

# Example usage
generator = DynamicShapeGenerator()
generator.create_dog(size=1.0)  # size parameter controls overall scale
dog_mesh = generator.generate()

# Visualize the result
o3d.visualization.draw_geometries([dog_mesh])

In [ ]:
import open3d as o3d
import numpy as np
from typing import Dict, List, Optional
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

class MLIntentClassifier:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.classifier = LinearSVC()
        self.intents = ['head', 'body', 'legs', 'ears', 'tail', 'face', 'collar']
        
        # Bulldog-specific training data
        mock_data = [
            ("short thick legs", 'legs'),
            ("wrinkled face", 'face'),
            ("stubby tail", 'tail'),
            ("broad chest", 'body'),
            ("floppy ears", 'ears'),
            ("leather collar", 'collar')
        ]
        texts, labels = zip(*mock_data)
        X = self.vectorizer.fit_transform(texts)
        self.classifier.fit(X, labels)

    def predict(self, text: str) -> str:
        X = self.vectorizer.transform([text])
        return self.classifier.predict(X)[0]

class BulldogShapeGenerator:
    @staticmethod
    def create_shape(shape_type: str, params: Dict) -> o3d.geometry.TriangleMesh:
        if shape_type == 'squashed_sphere':
            mesh = o3d.geometry.TriangleMesh.create_sphere(
                radius=params['radius'],
                resolution=params.get('resolution', 32)
            )
            vertices = np.asarray(mesh.vertices)
            vertices[:, 0] *= params.get('x_scale', 1.2)
            vertices[:, 1] *= params.get('y_scale', 1.0)
            vertices[:, 2] *= params.get('z_scale', 0.8)
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        elif shape_type == 'muscular_cylinder':
            mesh = o3d.geometry.TriangleMesh.create_cylinder(
                radius=params['radius'],
                height=params['height'],
                resolution=32
            )
            vertices = np.asarray(mesh.vertices)
            vertices[:, 2] = vertices[:, 2] * 0.8  # Shorten height
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        elif shape_type == 'wrinkled_surface':
            mesh = o3d.geometry.TriangleMesh.create_sphere(
                radius=params['radius'],
                resolution=params.get('resolution', 48)
            )
            vertices = np.asarray(mesh.vertices)
            noise = np.random.normal(0, 0.02, vertices.shape)
            vertices += noise
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        else:
            mesh = BulldogShapeGenerator._create_basic_shape(shape_type, params)
            
        mesh.compute_vertex_normals()
        return mesh

    @staticmethod
    def _create_basic_shape(shape_type: str, params: Dict):
        creators = {
            'sphere': o3d.geometry.TriangleMesh.create_sphere,
            'cylinder': o3d.geometry.TriangleMesh.create_cylinder,
            'cone': o3d.geometry.TriangleMesh.create_cone,
            'torus': o3d.geometry.TriangleMesh.create_torus
        }
        return creators[shape_type](**params)

class BulldogGenerator:
    def __init__(self):
        self.components = []
        self.reasoning_log = []
        self.intent_classifier = MLIntentClassifier()
        self.shape_generator = BulldogShapeGenerator()
        
        self.default_params = {
            'body_width': 1.5,
            'body_depth': 1.0,
            'leg_radius': 0.15,
            'head_radius': 0.7,
            'ear_length': 0.3,
            'tail_radius': 0.08
        }

    def log_step(self, message: str):
        self.reasoning_log.append(f"• {message}")

    def parse_description(self, text: str):
        self.log_step(f"Analyzing description: '{text}'")
        text = text.lower()
        
        # Detect features using classifier
        features = set()
        for word in text.split():
            try:
                features.add(self.intent_classifier.predict(word))
            except:
                continue
                
        self.log_step(f"Detected features: {', '.join(features)}")
        
        # Main body
        self._add_body_component(features)
        
        # Head and face
        self._add_head_component(features)
        
        # Limbs
        self._add_legs(features)
        self._add_tail(features)
        
        # Accessories
        if 'collar' in features:
            self._add_collar()

    def _add_body_component(self, features: set):
        body_params = {
            'shape_type': 'squashed_sphere',
            'params': {
                'radius': self.default_params['body_width']/2,
                'x_scale': 1.5,
                'z_scale': 0.6
            },
            'transform': np.eye(4)
        }
        self.components.append({'type': 'body', **body_params})
        self.log_step("Added muscular bulldog body")

    def _add_head_component(self, features: set):
        # Main head structure
        head_transform = np.eye(4)
        head_transform[0, 3] = self.default_params['body_width']/2 + 0.3
        head_params = {
            'shape_type': 'squashed_sphere' if 'wrinkled' in features else 'sphere',
            'params': {
                'radius': self.default_params['head_radius'],
                'z_scale': 0.7
            },
            'transform': head_transform
        }
        self.components.append({'type': 'head', **head_params})
        
        # Face features
        if 'face' in features:
            self._add_wrinkles(head_transform)
            self._add_eyes(head_transform)
            self._add_snout(head_transform)

    def _add_wrinkles(self, head_transform: np.ndarray):
        wrinkle_params = {
            'shape_type': 'wrinkled_surface',
            'params': {'radius': self.default_params['head_radius'] * 0.9},
            'transform': head_transform
        }
        self.components.append({'type': 'wrinkles', **wrinkle_params})
        self.log_step("Added facial wrinkles")

    def _add_eyes(self, head_transform: np.ndarray):
        eye_params = {
            'shape_type': 'sphere',
            'params': {'radius': 0.08},
            'transform': self._calculate_eye_transform(head_transform)
        }
        self.components.append({'type': 'eyes', **eye_params})

    def _calculate_eye_transform(self, base_transform: np.ndarray) -> np.ndarray:
        eye_transform = np.copy(base_transform)
        eye_transform[0, 3] += 0.2
        eye_transform[1, 3] += 0.15
        eye_transform[2, 3] += 0.1
        return eye_transform

    def _add_legs(self, features: set):
        leg_height = 0.4 if 'short' in features else 0.6
        positions = [
            (0.4, 0.4, -0.7),
            (-0.4, 0.4, -0.7),
            (0.4, -0.4, -0.7),
            (-0.4, -0.4, -0.7)
        ]
        
        for pos in positions:
            leg_transform = np.eye(4)
            leg_transform[:3, 3] = pos
            leg_params = {
                'shape_type': 'muscular_cylinder',
                'params': {
                    'radius': self.default_params['leg_radius'],
                    'height': leg_height
                },
                'transform': leg_transform
            }
            self.components.append({'type': 'leg', **leg_params})
        self.log_step(f"Added {len(positions)} sturdy legs")

    def _add_tail(self, features: set):
        tail_transform = np.eye(4)
        tail_transform[0, 3] = -self.default_params['body_width']/2 - 0.1
        tail_params = {
            'shape_type': 'cone',
            'params': {
                'radius': self.default_params['tail_radius'],
                'height': 0.2
            },
            'transform': tail_transform
        }
        self.components.append({'type': 'tail', **tail_params})
        self.log_step("Added stubby tail")

    def generate(self, description: str) -> o3d.geometry.TriangleMesh:
        self.components = []
        self.reasoning_log = []
        
        self.parse_description(description)
        
        full_mesh = o3d.geometry.TriangleMesh()
        for part in self.components:
            mesh = self.shape_generator.create_shape(part['shape_type'], part['params'])
            mesh.transform(part['transform'])
            full_mesh += mesh
        
        full_mesh.compute_vertex_normals()
        print("\n".join(self.reasoning_log))
        return full_mesh

# Example usage
generator = BulldogGenerator()
bulldog_mesh = generator.generate("A wrinkled bulldog with short legs and collar")

# Visualize the result
o3d.visualization.draw_geometries([bulldog_mesh],
                                  window_name="3D Bulldog",
                                  width=800,
                                  height=600)

In [ ]:
import open3d as o3d
import numpy as np
import random
# Example using Keras/TensorFlow for the RL Agent
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

class MLIntentClassifier:  # (Same as before)
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.classifier = LinearSVC()
        self.intents = ['head', 'body', 'legs', 'ears', 'tail', 'face', 'collar']
        
        # Bulldog-specific training data
        mock_data = [
            ("short thick legs", 'legs'),
            ("wrinkled face", 'face'),
            ("stubby tail", 'tail'),
            ("broad chest", 'body'),
            ("floppy ears", 'ears'),
            ("leather collar", 'collar')
        ]
        texts, labels = zip(*mock_data)
        X = self.vectorizer.fit_transform(texts)
        self.classifier.fit(X, labels)

    def predict(self, text: str) -> str:
        X = self.vectorizer.transform([text])
        return self.classifier.predict(X)[0]

class BulldogShapeGenerator: # (Same as before)
    @staticmethod
    def create_shape(shape_type: str, params: Dict) -> o3d.geometry.TriangleMesh:
        if shape_type == 'squashed_sphere':
            mesh = o3d.geometry.TriangleMesh.create_sphere(
                radius=params['radius'],
                resolution=params.get('resolution', 32)
            )
            vertices = np.asarray(mesh.vertices)
            vertices[:, 0] *= params.get('x_scale', 1.2)
            vertices[:, 1] *= params.get('y_scale', 1.0)
            vertices[:, 2] *= params.get('z_scale', 0.8)
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        elif shape_type == 'muscular_cylinder':
            mesh = o3d.geometry.TriangleMesh.create_cylinder(
                radius=params['radius'],
                height=params['height'],
                resolution=32
            )
            vertices = np.asarray(mesh.vertices)
            vertices[:, 2] = vertices[:, 2] * 0.8  # Shorten height
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        elif shape_type == 'wrinkled_surface':
            mesh = o3d.geometry.TriangleMesh.create_sphere(
                radius=params['radius'],
                resolution=params.get('resolution', 48)
            )
            vertices = np.asarray(mesh.vertices)
            noise = np.random.normal(0, 0.02, vertices.shape)
            vertices += noise
            mesh.vertices = o3d.utility.Vector3dVector(vertices)
        else:
            mesh = BulldogShapeGenerator._create_basic_shape(shape_type, params)
            
        mesh.compute_vertex_normals()
        return mesh

    @staticmethod
    def _create_basic_shape(shape_type: str, params: Dict):
        creators = {
            'sphere': o3d.geometry.TriangleMesh.create_sphere,
            'cylinder': o3d.geometry.TriangleMesh.create_cylinder,
            'cone': o3d.geometry.TriangleMesh.create_cone,
            'torus': o3d.geometry.TriangleMesh.create_torus
        }
        return creators[shape_type](**params)

class BulldogGenerator: # (Modified for RL)
    def __init__(self):
        self.components = []
        self.reasoning_log = []
        self.intent_classifier = MLIntentClassifier()
        self.shape_generator = BulldogShapeGenerator()
        
        self.default_params = {
            'body_width': 1.5,
            'body_depth': 1.0,
            'leg_radius': 0.15,
            'head_radius': 0.7,
            'ear_length': 0.3,
            'tail_radius': 0.08
        }

    def log_step(self, message: str):
        self.reasoning_log.append(f"• {message}")

    def parse_description(self, text: str):
        self.log_step(f"Analyzing description: '{text}'")
        text = text.lower()
        
        # Detect features using classifier
        features = set()
        for word in text.split():
            try:
                features.add(self.intent_classifier.predict(word))
            except:
                continue
                
        self.log_step(f"Detected features: {', '.join(features)}")
        
        # Main body
        self._add_body_component(features)
        
        # Head and face
        self._add_head_component(features)
        
        # Limbs
        self._add_legs(features)
        self._add_tail(features)
        
        # Accessories
        if 'collar' in features:
            self._add_collar()

    def _add_body_component(self, features: set):
        body_params = {
            'shape_type': 'squashed_sphere',
            'params': {
                'radius': self.default_params['body_width']/2,
                'x_scale': 1.5,
                'z_scale': 0.6
            },
            'transform': np.eye(4)
        }
        self.components.append({'type': 'body', **body_params})
        self.log_step("Added muscular bulldog body")

    def _add_head_component(self, features: set):
        # Main head structure
        head_transform = np.eye(4)
        head_transform[0, 3] = self.default_params['body_width']/2 + 0.3
        head_params = {
            'shape_type': 'squashed_sphere' if 'wrinkled' in features else 'sphere',
            'params': {
                'radius': self.default_params['head_radius'],
                'z_scale': 0.7
            },
            'transform': head_transform
        }
        self.components.append({'type': 'head', **head_params})
        
        # Face features
        if 'face' in features:
            self._add_wrinkles(head_transform)
            self._add_eyes(head_transform)
            self._add_snout(head_transform)

    def _add_wrinkles(self, head_transform: np.ndarray):
        wrinkle_params = {
            'shape_type': 'wrinkled_surface',
            'params': {'radius': self.default_params['head_radius'] * 0.9},
            'transform': head_transform
        }
        self.components.append({'type': 'wrinkles', **wrinkle_params})
        self.log_step("Added facial wrinkles")

    def _add_eyes(self, head_transform: np.ndarray):
        eye_params = {
            'shape_type': 'sphere',
            'params': {'radius': 0.08},
            'transform': self._calculate_eye_transform(head_transform)
        }
        self.components.append({'type': 'eyes', **eye_params})

    def _calculate_eye_transform(self, base_transform: np.ndarray) -> np.ndarray:
        eye_transform = np.copy(base_transform)
        eye_transform[0, 3] += 0.2
        eye_transform[1, 3] += 0.15
        eye_transform[2, 3] += 0.1
        return eye_transform

    def _add_legs(self, features: set):
        leg_height = 0.4 if 'short' in features else 0.6
        positions = [
            (0.4, 0.4, -0.7),
            (-0.4, 0.4, -0.7),
            (0.4, -0.4, -0.7),
            (-0.4, -0.4, -0.7)
        ]
        
        for pos in positions:
            leg_transform = np.eye(4)
            leg_transform[:3, 3] = pos
            leg_params = {
                'shape_type': 'muscular_cylinder',
                'params': {
                    'radius': self.default_params['leg_radius'],
                    'height': leg_height
                },
                'transform': leg_transform
            }
            self.components.append({'type': 'leg', **leg_params})
        self.log_step(f"Added {len(positions)} sturdy legs")

    def _add_tail(self, features: set):
        tail_transform = np.eye(4)
        tail_transform[0, 3] = -self.default_params['body_width']/2 - 0.1
        tail_params = {
            'shape_type': 'cone',
            'params': {
                'radius': self.default_params['tail_radius'],
                'height': 0.2
            },
            'transform': tail_transform
        }
        self.components.append({'type': 'tail', **tail_params})
        self.log_step("Added stubby tail")

    def generate(self, description: str, params_override=None) -> o3d.geometry.TriangleMesh:
        self.components = []
        self.reasoning_log = []
        
        self.parse_description(description)
        
        full_mesh = o3d.geometry.TriangleMesh()
        for part in self.components:
            # Apply parameter overrides if provided
            if params_override and part['type'] in params_override:
                mesh = self.shape_generator.create_shape(part['shape_type'], params_override[part['type']])
            else:
                mesh = self.shape_generator.create_shape(part['shape_type'], part['params'])

            mesh.transform(part['transform'])
            full_mesh += mesh
        
        full_mesh.compute_vertex_normals()
        print("\n".join(self.reasoning_log))
        return full_mesh

    def get_state(self):
        """Returns the current state of the bulldog (e.g., parameters)."""
        # This is a simplified example.  A real state would include more features.
        return {
            'body_width': self.default_params['body_width'],
            'body_depth': self.default_params['body_depth'],
            'leg_radius': self.default_params['leg_radius'],
            'head_radius': self.default_params['head_radius']
        }

    def apply_action(self, action):
        """Applies an action to modify the bulldog parameters."""
        # Action is a dictionary of parameter changes.  Example: {'body_width': 0.1, 'leg_radius': -0.05}
        for param, change in action.items():
            self.default_params[param] += change
            # Clamp parameters to reasonable ranges (important!)
            self.default_params[param] = max(0.1, min(self.default_params[param], 2.0))  # Example clamping

# --------------------------------------------------------------------------------
# RL Agent (DQN Example)
# --------------------------------------------------------------------------------

def create_q_model(num_states, num_actions):
    """Creates a simple Q-network model."""
    inputs = layers.Input(shape=(num_states,))
    layer1 = layers.Dense(32, activation="relu")(inputs)
    layer2 = layers.Dense(32, activation="relu")(layer1)
    outputs = layers.Dense(num_actions, activation="linear")(layer2)  # Linear activation for Q-values
    return tf.keras.Model(inputs=inputs, outputs=outputs)

class RLAgent:
    def __init__(self, num_states, num_actions):
        self.num_states = num_states
        self.num_actions = num_actions
        self.model = create_q_model(num_states, num_actions)
        self.target_model = create_q_model(num_states, num_actions)  # Target network for stability
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.gamma = 0.99  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_decay = 0.001
        self.epsilon_min = 0.1

    def get_action(self, state):
        """Chooses an action based on the current state (epsilon-greedy)."""
        if random.random() < self.epsilon:
            return random.randrange(self.num_actions)  # Explore
        else:
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            q_values = self.model(state_tensor)
            return tf.argmax(q_values[0]).numpy()  # Exploit

    def train(self, state, action, reward, next_state, done):
        """Trains the Q-network using a single experience tuple."""
        with tf.GradientTape() as tape:
            q_values = self.model(tf.expand_dims(state, 0))
            target_q_values = self.target_model(tf.expand_dims(next_state, 0))
            
            # Calculate target Q-value (Bellman equation)
            if done:
                target = reward
            else:
                target = reward + self.gamma * tf.reduce_max(target_q_values)

            # Calculate the loss (mean squared error)
            q_value = q_values[0][action]
            loss = tf.keras.losses.MeanSquaredError()(tf.expand_dims(target, 0), tf.expand_dims(q_value, 0))

        # Backpropagation
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        # Update exploration rate
        self.epsilon = max(self.epsilon_min, self.epsilon - self.epsilon_decay)

    def update_target_model(self):
        """Updates the target network with the weights of the main network."""
        self.target_model.set_weights(self.model.get_weights())

# --------------------------------------------------------------------------------
# Reward Function (Example)
# --------------------------------------------------------------------------------

def reward_function(bulldog_mesh):
    """
    Calculates a reward based on how "realistic" or "appealing" the bulldog is.
    This is a placeholder and needs to be replaced with a more sophisticated approach.
    """
    # Example: Reward based on a simple metric like surface area.  More realistic
    # bulldogs might have a surface area within a certain range.
    surface_area = bulldog_mesh.get_surface_area()
    if 5 < surface_area < 7:
        reward = 1.0
    else:
        reward = -0.1

    # Add more sophisticated reward signals here, potentially using:
    # 1.  Pre-trained image classifiers to evaluate "bulldog-ness"
    # 2.  User feedback (if available)
    # 3.  Metrics related to mesh quality (e.g., smoothness, curvature)

    return reward

# --------------------------------------------------------------------------------
# Training Loop
# --------------------------------------------------------------------------------

# Define the action space (possible parameter changes)
ACTIONS = {
    0: {'body_width': 0.05},
    1: {'body_width': -0.05},
    2: {'leg_radius': 0.01},
    3: {'leg_radius': -0.01},
    4: {'head_radius': 0.02},
    5: {'head_radius': -0.02}
}
NUM_ACTIONS = len(ACTIONS)

# Initialize the environment and agent
generator = BulldogGenerator()
NUM_STATES = len(generator.get_state())  # Number of parameters in the state
agent = RLAgent(NUM_STATES, NUM_ACTIONS)

# Training parameters
NUM_EPISODES = 100
UPDATE_TARGET_FREQUENCY = 5  # Update target network every N episodes

for episode in range(NUM_EPISODES):
    # Reset the environment for each episode
    description = "A normal bulldog" #keep description constant or randomly generate
    generator = BulldogGenerator()
    state = list(generator.get_state().values()) # get the initial state

    done = False
    total_reward = 0

    while not done:
        # Choose an action
        action = agent.get_action(state)

        # Apply the action to the environment
        action_to_apply = ACTIONS[action]
        generator.apply_action(action_to_apply)

        # Generate the bulldog mesh
        bulldog_mesh = generator.generate(description)

        # Get the reward
        reward = reward_function(bulldog_mesh)
        total_reward += reward

        # Get the next state
        next_state = list(generator.get_state().values())

        # Check if the episode is done (e.g., reached a maximum number of steps)
        # You can define a termination condition based on the bulldog's parameters.
        # For example, terminate if the body width becomes too large or too small.
        if generator.default_params['body_width'] > 2.0 or generator.default_params['body_width'] < 0.5:
            done = True

        # Train the agent
        agent.train(np.array(state), action, reward, np.array(next_state), done)

        # Update the state
        state = next_state

    # Update target network
    if episode % UPDATE_TARGET_FREQUENCY == 0:
        agent.update_target_model()

    print(f"Episode {episode + 1}: Total Reward = {total_reward}, Epsilon = {agent.epsilon}")

print("Training complete!")

# Example usage after training:
# Generate a bulldog with the learned parameters. The generator now contains the 'optimized' parameters in default_params.
bulldog_mesh = generator.generate("A trained bulldog")
o3d.visualization.draw_geometries([bulldog_mesh],
                                  window_name="3D Bulldog",
                                  width=800,
                                  height=600)
